In [1]:
#check goldfinger and goldeneye.
#why is bond not protagonist?
#kaggle data is not reliable in terms of character importance, need another method to select protagonists

from llm_annotate import chunk_text, annotate, custom_openai
import pandas as pd
import os

In [2]:
#load kaggle data
kaggle_data = pd.read_csv("data/kaggle_data.csv")

In [3]:
repeat_chunking = False
if repeat_chunking:
    for fn in kaggle_data.script_filename:
        with open(f"data/screenplay_data/data/raw_texts/raw_texts/{fn}", "r", encoding="utf-8") as f:
            try:
                script = f.read()
            except Exception as e:
                try:
                    with open(f"data/screenplay_data/data/raw_texts/raw_texts/{fn}", "r", encoding="iso-8859-1") as f:
                        script = f.read()
                except Exception as e2:
                    print(f"again Error reading {fn}: {e2}")
                    raise e2
        chunk_text(script, outputfile=f"data/chunks/{fn.replace('.txt', '.json')}", verbose=False)

In [4]:
traits = {
    "warmth": {
        "trait_explanation": "A character’s warmth refers to their tendency to be good-natured, trustworthy, tolerant, friendly, and sincere.",
        "examples": [
            {"name": "John Doe",
             "action": "Organizes a welcome party for a new neighbor.",
             "assessment": "Organizing a welcome party is a thoughtful and inclusive gesture.",
             "rating": 3},
            {"name": "Jane Smith",
             "action": "Tells Lisa that her dress is ugly.",
             "assessment": "In this particular scene, the character seemed rather unaware of her friends feelings and her need for benevolent guidance.",
             "rating": -2}
        ]
    },
    "competence": {
        "trait_explanation": "A character’s competence refers to their tendency to behave capably, skillfully, intelligently, and confidently.",
        "examples": [
            {"name": "John Doe",
             "action": "Trips and falls in front of his crush.",
             "assessment": "This action suggests a slight lack of coordination, although it might be specific to this particular situation.",
             "rating": -1},
            {"name": "Jane Smith",
             "action": "Takes out two armed guards without breaking a sweat.",
             "assessment": "This action indicates that Jane's physical skills and dominance are far above average.",
             "rating": 3}
        ]
    }
}

In [9]:
def annotate_one(i):
    current_file = kaggle_data.script_filename.iloc[i]
    out_path = f"data/annotations/{current_file.replace('.txt', '.json')}"
    if os.path.exists(out_path):
        return f'Skipping {i} '
    print(f"{i+1}/{kaggle_data.shape[0]}")
    title = kaggle_data.title.iloc[i]
    protagonist = kaggle_data.character_name_search.iloc[i]
    annotate(
        chunkfile=f"data/chunks/{current_file.replace('.txt', '.json')}",
        outputfile=out_path,
        book_title=title,
        traits=traits,
        target_characters=[protagonist],
        model="gpt-4.1-mini"
    )
    return f"Done {i}: {current_file}"

In [ ]:
#(x - 64.12 ) * 440
from concurrent.futures import ThreadPoolExecutor, as_completed
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(annotate_one, i) for i in range(500)]  # or range(kaggle_data.shape[0])
    for future in as_completed(futures):
        print(future.result())

Skipping 4 
Skipping 0 
Skipping 6 
Skipping 2 
Skipping 3 
Skipping 1 
Skipping 7 
Skipping 5 
Skipping 10 
Skipping 13 
Skipping 14 
Skipping 8 
Skipping 15 
Skipping 9 
Skipping 16 
Skipping 12 
Skipping 17 
Skipping 18 
Skipping 19 
Skipping 11 
Skipping 20 
Skipping 25 
Skipping 22 
Skipping 23 
Skipping 24 
Skipping 21 
Skipping 26 
Skipping 27 
41/2138
Skipping 28 
Skipping 29 
Skipping 30 
Skipping 31 
Skipping 32 
Skipping 37 
Skipping 38 
Skipping 34 
Skipping 36 
43/2138
Skipping 33 
Skipping 35 
45/2138
42/2138
44/2138
Skipping 39 
Total annotations collected: 271
Done 44: A Fantastic Woman_5639354.txt
Total annotations collected: 302
Done 40: A Clockwork Orange_0066921.txt
Total annotations collected: 367
Done 41: A Dangerous Method_1571222.txt
Total annotations collected: 368
Done 42: A Death in the Gunj_5918074.txt
Total annotations collected: 634
Done 43: A Dry White Season_0097243.txt


In [ ]:
from llm_annotate import score_annotations

score_annotations(annotation_file="data/annotations/10 Things I Hate About You_0147800.json", chunkfile = "data/chunks/10 Things I Hate About You_0147800.json", outputfile="trash.jsonl")

In [ ]:
#PLOT DISTRIBUTION OF binary columns Drama,Thriller,Comedy,Action,Crime,Romance,Adventure,Sci-Fi,Mystery,Horror,Fantasy,Biography
import matplotlib.pyplot as plt
genre_cols = ["Drama","Thriller","Comedy","Action","Crime","Romance","Adventure","Sci-Fi","Mystery","Horror","Fantasy","Biography"]
genre_counts = kaggle_data[genre_cols].sum().sort_values(ascending=False)
plt.figure(figsize=(10,6))
genre_counts.plot(kind='bar')
plt.title("Distribution of Movie Genres in Kaggle Dataset")
plt.xlabel("Genre")
plt.ylabel("Number of Movies")
plt.tight_layout()
